<a href="https://colab.research.google.com/github/basugautam/Reproducibility-Challenge-Project/blob/main/3_user_defined_constraints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# 🔹 Step 1: Load Dataset
dataset_path = r"C:\LCTSF\Dataset\timeseries_dataset.csv"  # Adjust this if necessary
df = pd.read_csv(dataset_path)

# 🔹 Step 2: Preprocessing
# Assume the dataset contains a single time series column named 'value'
data = df['value'].values.reshape(-1, 1)  # Reshape for scaling

# Normalize data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# 🔹 Step 3: Create Sequences for Supervised Learning
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, 0])  # Input sequence
        y.append(data[i+seq_length, 0])    # Target output
    return np.array(X), np.array(y)

sequence_length = 50  # Define window size (time steps)
X, y = create_sequences(data_scaled, sequence_length)

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Reshape data for LSTM input format
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# 🔹 Step 4: Define Custom Constrained Loss Function
def constrained_loss(y_true, y_pred, max_error=0.05):
    """
    Custom loss function that penalizes large forecast errors.

    Parameters:
    - y_true: True target values
    - y_pred: Predicted values
    - max_error: Upper bound constraint for acceptable error

    Returns:
    - Modified loss that enforces constraints
    """
    # Calculate absolute error
    error = tf.abs(y_true - y_pred)

    # Apply penalty for errors exceeding max_error constraint
    penalty = tf.where(error > max_error, (error - max_error) ** 2, 0.0)

    # Compute final loss as mean of constrained errors
    return tf.reduce_mean(error + penalty)

# 🔹 Step 5: Build LSTM Model
model = Sequential([
    LSTM(units=64, return_sequences=False, input_shape=(sequence_length, 1)),
    Dropout(0.2),  # Prevent overfitting
    Dense(units=1)  # Output layer
])

# Compile model using custom constrained loss function
model.compile(optimizer=Adam(learning_rate=0.001), loss=constrained_loss, metrics=['mae'])

# 🔹 Step 6: Train Model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# 🔹 Step 7: Plot Training Loss with Constraints
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Constrained Model Loss During Training')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 🔹 Step 8: Make Predictions
y_pred = model.predict(X_test)

# Invert scaling for original values
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# 🔹 Step 9: Visualize Actual vs. Predicted Values
plt.figure(figsize=(10,6))
plt.plot(y_test, label='Actual Values')
plt.plot(y_pred, label='Predicted Values (Constrained)')
plt.title('Forecasting with Constrained Error')
plt.xlabel('Time Steps')
plt.ylabel('Value')
plt.legend()
plt.show()

# 🔹 Step 10: Evaluate Model Performance
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\LCTSF\\Dataset\\timeseries_dataset.csv'